In [1]:
from casadi import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [128]:
def opti_steps(m_l=0.5, k=3000, tau_max=0.05, omega_max=3000, nsteps=3, t=0.2, X_init=[0,0,0], T_guess=[0.0003, 5]):
    ''' Optimize T1 and T2 over multiple steps '''

    ''' Function for one analytical step '''
   
    X0 = SX.sym('X0', 5) #states: x_a, x_l, v_l, T1_last, T2_last
    x_a0 = X0[0]
    x_l0 = X0[1]
    v_l0 = X0[2]
    T1_0 = X0[3]
    T2_0 = X0[4]
    T = SX.sym('T', 2) #transmission ratio for this step
    T1 = T[0]
    T2 = T[1]
        # X[1] = Ts_opt[1,i]*((Ts_opt[0,i]-T[0])*X[0] + X[1]/T[1])
    x_l0 = T2*(x_a0*(T1-T1_0) + x_l0/T2_0)

    x0 = 1/T2
    x1 = T2**2
    x2 = omega_max**2
    x3 = x1*x2
    x4 = k**2
    x5 = m_l**2
    x6 = T1**4*x5
    x7 = x4*x6
    x9 = 1/tau_max
    x10 = (1/2)*x9
    x12 = T1**2
    x13 = omega_max*t
    x14 = k*x10*x12*x13
    x15 = exp(x14)
    x16 = tau_max*x15
    x17 = x16*x_a0
    x18 = T1*omega_max
    x19 = T2*m_l*v_l0*x18
    x20 = m_l*x12
    x21 = x20*x3
    x22 = k*m_l
    x23 = 2*tau_max
    x24 = x23*x_l0
    x25 = x12*x22
    x26 = x23*x_a0
    x27 = T2*omega_max
    x28 = T2**3
    x29 = omega_max**3
    x30 = T1**3
    x32 = exp(-x14)
    x33 = x32*x9
    x34 = x1*x18
    x35 = T2*v_l0 - x34
    x36 = 2*k
    x37 = T1*T2
    x38 = omega_max*x1
    x39 = 2*v_l0*x25*x38
    x40 = x2*x28
    x41 = x22*x30*x40
    x42 = 2*x41
    x43 = tau_max*x36
    x44 = -x37*x43*x_a0 + x43*x_l0
    r = 4*k*m_l*tau_max**2 - x3*x7
    R_ad_re = if_else(r>0, sqrt(r), 0)
    R_ad_im = if_else(r>0, 0, sqrt(-r))
    x11_re = t*x0*x10*R_ad_re/m_l
    x11_im = t*x0*x10*R_ad_im/m_l
    C_os = cos(x11_re)*cosh(x11_im)
    S_in_re = sin(x11_re)*cosh(x11_im)
    S_in_im = cos(x11_re)*sinh(x11_im)
    SdR = (S_in_re*R_ad_re + S_in_im*R_ad_im) / (R_ad_re**2 + R_ad_im**2) # S_in/R_ad should be purely real (aka x31)
    SmR = S_in_re*R_ad_re - S_in_im*R_ad_im  # S_in*R_ad should be purely real
    x_a = x33*(C_os*(-x19 + x21) + x13*x16 + x15*x19 - x15*x21 + x17 + SdR*(-k*v_l0*x3*x30*x5 + k*x28*x29*x6 + x18*x22*x24 - x25*x26*x27))
    x_l = (1/2)*x33*(C_os*(-x39 + x42 + x44) + SmR*x35 + T2*t*x16*x18*x36 + x15*x39 - x15*x42 + x17*x36*x37 + SdR*(T1**5*T2**4*x29*x4*x5 - m_l*x26*x30*x38*x4 - v_l0*x40*x7 + x20*x24*x27*x4))/k
    v_l = x0*x32*(C_os*x35 + x15*x34 + SdR*(k*m_l*omega_max*v_l0*x1*x12 - x41 - x44))
    X_new = SX.sym('X_new', 5)
    X_new[0] = x_a
    X_new[1] = x_l
    X_new[2] = v_l
    X_new[3] = T1
    X_new[4] = T2

    Fstep = Function("step", [X0, T], [X_new])
    Fstep_n = Fstep.mapaccum(nsteps)


    ''' Optimization '''
    opti = Opti()

    Ts = opti.variable(2, nsteps) #column vectors of [T1, T2], concat horizontally
    X_n = Fstep_n(np.concatenate((X_init, T_guess)), Ts)

    opti.minimize(-X_n[-1,2])
    opti.subject_to(opti.bounded(1e-6, Ts[0,:], 0.001))
    opti.subject_to(opti.bounded(1e-6, Ts[1,:], 20))
    opti.set_initial(Ts[0,:], T_guess[0])
    opti.set_initial(Ts[1,:], T_guess[1])

    log = []
    opti.callback(lambda i: log.append(opti.debug.value(X_n)))

    opts = {'ipopt.print_level':0, 'print_time':0}
    opti.solver('ipopt', opts)
    sol = opti.solve()

    return sol, np.array(log), opti, X_n

In [125]:
print(log)

[[[ 3.74873467e+01  7.49493999e+01  1.12386181e+02 ...  1.40715010e+03
    1.44371756e+03  1.48026269e+03]
  [ 1.17166995e-06  9.37160252e-06  3.16228349e-05 ...  6.33199506e-02
    6.84072579e-02  7.37567083e-02]
  [ 2.81183801e-04  1.12444707e-03  2.52930517e-03 ...  3.96664500e-01
    4.17387615e-01  4.38606896e-01]
  [ 3.00000000e-04  3.00000000e-04  3.00000000e-04 ...  3.00000000e-04
    3.00000000e-04  3.00000000e-04]
  [ 5.00000000e+00  5.00000000e+00  5.00000000e+00 ...  5.00000000e+00
    5.00000000e+00  5.00000000e+00]]

 [[ 3.74412851e+01  7.47658904e+01  1.11974201e+02 ...  1.34492799e+03
    1.37831135e+03  1.41160118e+03]
  [ 2.52332311e-06 -2.37150658e-04 -1.89340467e-04 ...  1.32832701e-01
    1.43439768e-01  1.54584596e-01]
  [ 6.05436897e-04  2.41777955e-03  5.43178966e-03 ...  8.27377803e-01
    8.69914721e-01  9.13418137e-01]
  [ 6.46504950e-04  6.45130567e-04  6.45130567e-04 ...  6.45130567e-04
    6.45130567e-04  6.45130567e-04]
  [ 5.00018465e+00  5.00042992e+00 

In [127]:
m_l=0.5
k=10
tau_max=0.05
omega_max=3000
t_total = 0.5


def get_step(X0=[0,0,0], Ts=[0.0003, 20]):
    x_a0, x_l0, v_l0 = X0
    T1, T2 = Ts
    x0 = 1/T2
    x1 = T2**2
    x2 = omega_max**2
    x3 = x1*x2
    x4 = k**2
    x5 = m_l**2
    x6 = T1**4*x5
    x7 = x4*x6
    x9 = 1/tau_max
    x10 = (1/2)*x9
    x12 = T1**2
    x13 = omega_max*t
    x14 = k*x10*x12*x13
    x15 = exp(x14)
    x16 = tau_max*x15
    x17 = x16*x_a0
    x18 = T1*omega_max
    x19 = T2*m_l*v_l0*x18
    x20 = m_l*x12
    x21 = x20*x3
    x22 = k*m_l
    x23 = 2*tau_max
    x24 = x23*x_l0
    x25 = x12*x22
    x26 = x23*x_a0
    x27 = T2*omega_max
    x28 = T2**3
    x29 = omega_max**3
    x30 = T1**3
    x32 = exp(-x14)
    x33 = x32*x9
    x34 = x1*x18
    x35 = T2*v_l0 - x34
    x36 = 2*k
    x37 = T1*T2
    x38 = omega_max*x1
    x39 = 2*v_l0*x25*x38
    x40 = x2*x28
    x41 = x22*x30*x40
    x42 = 2*x41
    x43 = tau_max*x36
    x44 = -x37*x43*x_a0 + x43*x_l0
    r = 4*k*m_l*tau_max**2 - x3*x7
    R_ad_re = sqrt(r) if r > 0 else 0
    R_ad_im = 0 if r>0 else sqrt(-r)
    x11_re = t*x0*x10*R_ad_re/m_l
    x11_im = t*x0*x10*R_ad_im/m_l
    C_os = cos(x11_re)*cosh(x11_im)
    S_in_re = sin(x11_re)*cosh(x11_im)
    S_in_im = cos(x11_re)*sinh(x11_im)
    SdR = (S_in_re*R_ad_re + S_in_im*R_ad_im) / (R_ad_re**2 + R_ad_im**2) # S_in/R_ad should be purely real (aka x31)
    SmR = S_in_re*R_ad_re - S_in_im*R_ad_im  # S_in*R_ad should be purely real
    x_a = x33*(C_os*(-x19 + x21) + x13*x16 + x15*x19 - x15*x21 + x17 + SdR*(-k*v_l0*x3*x30*x5 + k*x28*x29*x6 + x18*x22*x24 - x25*x26*x27))
    x_l = (1/2)*x33*(C_os*(-x39 + x42 + x44) + SmR*x35 + T2*t*x16*x18*x36 + x15*x39 - x15*x42 + x17*x36*x37 + SdR*(T1**5*T2**4*x29*x4*x5 - m_l*x26*x30*x38*x4 - v_l0*x40*x7 + x20*x24*x27*x4))/k
    v_l = x0*x32*(C_os*x35 + x15*x34 + SdR*(k*m_l*omega_max*v_l0*x1*x12 - x41 - x44))

    return [x_a, x_l, v_l]

# step_array = np.concatenate((range(2, 20, 1), range(20, 51, 2)))
step_array = [40]
for nsteps in step_array:

    t = t_total/nsteps

    sol, log, opti, X_n = opti_steps(m_l=m_l, k=k, tau_max=tau_max, omega_max=omega_max, nsteps=nsteps, t=t)
    
    #simulate optimal sol
    Ts_opt = np.insert(log[-1], 0, [log[-1][0,0],log[-1][1,0]], axis=1)

    X = [0,0,0]
    log_X = [X]
    log_spring = [X[0]*T[0] - X[1]/T[1]]
    T = Ts_opt[0]
    for i in range(nsteps):
        X[1] = Ts_opt[1,i]*((Ts_opt[0,i]-T[0])*X[0] + X[1]/T[1])
        T = Ts_opt[:,i]
        X = get_step(X, T)
        log_X.append(X)
        log_spring.append(X[0]*T[0] - X[1]/T[1])
    log_X = np.array(log_X)

    print(f"{nsteps}: v_l_cas={sol.value(X_n[2])}, v_l_for={log_X[-1][2]}")

    fig, axs = plt.subplots(6, 1, sharex=True, figsize=(8,8))
    ts = np.linspace(0, t_total, nsteps+1)
    axs[0].set_title(f"Optimal transmission ratios, {nsteps} steps over {round(t_total, 2)} sec.")

    # axs[0].plot(ts, log_X[:,0], 'r.-')
    log_X_derivative = np.gradient(log_X[:,0], ts)
    axs[0].plot(ts, log_X_derivative, 'g.-')
    axs[0].set_ylabel('v_a')

    axs[1].plot(ts, log_X[:,1], 'r.-')
    axs[1].set_ylabel('x_l')

    axs[2].plot(ts, log_X[:,2], 'g.-')
    axs[2].set_ylabel('v_l')

    axs[3].plot(ts, log_spring, 'y.-')
    axs[3].plot(ts, np.zeros_like(log_spring), 'k--')
    axs[3].set_ylabel('spring')

    axs[4].step(ts, Ts_opt[0])
    axs[4].set_ylim(0, max(Ts_opt[0])*1.05)
    axs[4].set_ylabel("T1")
    axs[4].ticklabel_format(axis='y', style='sci', scilimits=(0,0))


    axs[5].step(ts, Ts_opt[1])
    axs[5].set_ylim(0, max(Ts_opt[1])*1.05)
    axs[5].set_ylabel("T2")

    plt.savefig(f"profilesteps/{round(t_total*1000):04d}ms_{nsteps:04d}steps.png")


ValueError: could not broadcast input array from shape (2,1) into shape (5,1)

In [102]:
log_spring

[0.0,
 0.03612811664747916,
 0.0696429679591227,
 0.10073086176901856,
 0.12956482470057645,
 0.15630555000352242,
 0.18110227656419353,
 0.20409360505129592,
 0.22540825577445905,
 0.24516577297255446,
 0.2634771820018044,
 0.28049665543980973,
 0.2965316873904541,
 0.3117254554801797,
 0.32618922791600746,
 0.34001125896068873,
 0.3532628107713364,
 0.3660023268840061,
 0.378278392321779,
 0.3901318789594974,
 0.40159753134181986,
 0.412705159953762,
 0.4234805536411239,
 0.4339461874560457,
 0.44412177903011585,
 0.4540247311094745,
 0.46367048736456906,
 0.47307282130342776,
 0.4822440729923686,
 0.4911953446274222,
 0.49993666334865433,
 0.5084771177444243,
 0.516824973048556,
 0.5249877689491453,
 0.5329724031053816,
 0.5407852028388085,
 0.5484319869787098,
 0.5559181194618978,
 0.5632485559891401,
 0.5704278848045986,
 0.5774603624766828]

In [40]:
end_v

5.456247516765989